In [5]:
import pandas as pd

In [6]:
#Load the structured data 
df = pd.read_excel('CSE_to_convert.xlsx', sheet_name= 'sheet1')

In [ ]:
df.columns

In [ ]:
data= pd.read_excel('/Users/ludwingmoncadabellorin/Downloads/nic_ds/USD_COR BCN.xlsx')
# Create Python dictionary where the keys are the 'year' column of a pandas DataFrame named 'data' 
#and the values are the 'rate' column of the same DataFrame
exchange_rates = dict(zip(data['year'], data['rate']))

#Double Check US$ and C$ are the only currency symbols
currencies = df['Monto'].str[:2].unique()
currencies

In [10]:
# Split the Monto column into currency and amount columns
df[['moneda', 'monto']] = df['Monto'].str.split(n=1, expand=True) 

In [11]:
# Delete commas from values in 'monto'
df['monto'] = pd.to_numeric(df['monto'].str.replace(',', ''), errors='coerce')

In [12]:
# Convert 'Monto' column to numeric type
df['monto'] = pd.to_numeric(df['monto'], errors='coerce')

In [13]:
def convert_currency(value, currency, year, exchange_rates):
    """
    Convert a monetary value from the original currency to C$ using the exchange rate
    for the given year.

    Args:
        value (float or int): The monetary value to convert.
        currency (str): The currency symbol for the original currency.
        year (int): The year of the exchange rate to use for the conversion.
        exchange_rates (dict): A dictionary containing exchange rates for each year.

    Returns:
        float: The converted monetary value in C$, or the original value if the currency
               is already in C$.

    Raises:
        ValueError: If the value is not a float or integer, the currency symbol is not
                    recognized, or there is no exchange rate available for the given year.
    """
    if not isinstance(value, (int, float)):
        raise ValueError(f"Invalid value: {value}")
    elif currency == 'C$':
        return value
    elif currency == 'US$':
        exchange_rate = exchange_rates.get(year, None)
        if exchange_rate:
            return value * exchange_rate
        else:
            raise ValueError(f"No exchange rate found for year {year}")
    else:
        raise ValueError(f"Invalid currency symbol: {currency}")

In [ ]:
df['Fecha_Publicacion'] = pd.to_datetime(df['Fecha_Publicacion'])


In [15]:
df['Fecha_Publicacion'] = pd.to_datetime(df['Fecha_Publicacion'], format='%d/%m/%Y')


In [16]:
# Apply currency conversion to the Amount column
df['monto'] = df.apply(lambda row: convert_currency(row['monto'], row['moneda'], 
                                                    row['Fecha_Publicacion'].year, exchange_rates), axis=1)


In [ ]:
df.head()

In [17]:
# Drop moneda, since we do not need it anymore 
df = df.drop('moneda', axis=1)

In [18]:
#Let's rename monto to converted_monto
df = df.rename(columns={'monto': 'converted_monto'})

In [19]:
df.to_excel('cse_converted_amount.xlsx', index=False)

In [ ]:
df.head()